# Soda scan with checks and profiling on delta table

## Setup

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path 
import pandas as pd
from utils.data_generation import FakerProfileDataSnapshot

from pyspark.sql import SparkSession
from  pyspark.sql.functions import input_file_name, split, regexp_extract

In [3]:
from delta import *

In [4]:
from pyspark.sql import SparkSession
from delta import *

builder = SparkSession.builder.appName("data-pipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
# object for data generation
datagen = FakerProfileDataSnapshot(
    data_dir=str(Path('./data').resolve())
)
# clean landing storage
datagen.delete_batches_in_landing_dir()
print(datagen.bronze_dir)
datagen.delete_bronze_dir()

/home/jovyan/work/data/bronze


## Parameters

In [6]:
read_csv_options = {'header': 'true', 'inferSchema': 'true', 'delimiter': ',', 'quote': '"', 'multiLine': 'true'}

In [7]:
location_landing = datagen.landing_dir
location_bronze = datagen.bronze_dir

## Processing

### Generate data - CSV

In [8]:
path_csv = datagen.land_batch(7)
path_csv

'/home/jovyan/work/data/landing/snapshot_2022-09-30.csv'

### Create delta table 

In [9]:
df = spark.read.format('csv').options(**read_csv_options).load(str(path_csv)) \
    .withColumn("_delivery", regexp_extract(input_file_name(), '.*_(.*).csv$', 1))
df.toPandas()

,id,job,company,ssn,website,username,name,address,mail,birthdate,_delivery
0,0,Contractor,"Boone, Gallagher and Scott",508-98-7365,"['https://hall.info/', 'https://www.monroe-haw...",paula86,Misty Phillips,"4978 Chapman Bypass\nSanchezfurt, TN 23177",ana51@yahoo.com,1996-12-15,2022-09-30
1,1,Rural practice surveyor,Osborne PLC,861-51-6071,"['https://kelley.net/', 'http://www.herring-ca...",andreawhite,Alfred Hall,"11070 Wright Creek Apt. 541\nEast Jonathan, TN...",shawn57@yahoo.com,1922-02-28,2022-09-30
2,3,Newspaper journalist,Gordon-Smith,627-52-4610,['http://www.jackson.com/'],matthew14,Kyle Randall,"98150 Jones Way Apt. 251\nJonesside, OK 11215",michael49@hotmail.com,1922-03-06,2022-09-30
3,4,"Programmer, systems",Rodriguez-Williams,303-02-0239,"['https://www.pearson.com/', 'http://carlson.c...",kdavis,Sally Davis,"4052 Sparks Prairie\nJohnville, RI 76389",sanchezandrew@gmail.com,1961-04-20,2022-09-30
4,6,"Surveyor, rural practice",Jackson-Davis,498-01-2910,"['https://evans.info/', 'http://www.choi.com/']",simpsonphillip,Keith Todd,"280 Lindsey Road Apt. 412\nSouth Melissabury, ...",umartin@yahoo.com,1923-10-24,2022-09-30
5,9,"Surveyor, commercial/residential",Kelly-Singh,402-72-6549,"['http://www.english.com/', 'https://wilson.co...",rpeterson,Kelsey Ramsey,"36377 Christensen Fort\nPort Justin, OR 51565",shannonscott@yahoo.com,1987-05-30,2022-09-30
6,12,Secondary school teacher,Hernandez Ltd,334-42-9390,['http://walker.biz/'],harriscarl,Veronica Maldonado,"32860 Yoder Oval\nJeffreyborough, NM 31331",xchavez@hotmail.com,1932-04-26,2022-09-30
7,13,Race relations officer,"Jones, Randall and Cherry",418-26-0136,"['https://turner-hunter.org/', 'http://www.fie...",hbryant,Samuel Mack,"2024 Hodge Radial\nStefaniechester, NY 38154",bvazquez@yahoo.com,2012-05-14,2022-09-30
8,14,"Radiographer, diagnostic",Baldwin Group,412-92-6966,"['https://shah.net/', 'https://www.cohen-white...",gphillips,Anita Price,"97126 Gary Glens Suite 847\nPort Katherine, NM...",toddjessica@yahoo.com,2003-02-01,2022-09-30
9,15,"Geologist, engineering",Fernandez Group,308-98-1651,"['https://morgan.com/', 'https://www.gordon.co...",robert00,Courtney Gonzalez,"34549 Johnson Isle\nDonaldview, WI 19280",reedchristopher@gmail.com,1947-06-13,2022-09-30


In [10]:
df.write.format('delta').option('overwriteSchema', 'true').mode('overwrite').save(location_bronze)

In [11]:
spark.sql('DROP TABLE IF EXISTS snapshot')
spark.sql(f'CREATE TABLE IF NOT EXISTS snapshot USING DELTA LOCATION "{location_bronze}"')

DataFrame[]

In [12]:
spark.sql(f'DESCRIBE HISTORY snapshot').toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,0,2022-08-15 19:12:32.553,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '14', 'numOutputBytes': '628...",None,Apache-Spark/3.2.1 Delta-Lake/2.0.0


### Run soda scan

... with checks, discovery, profiling.

In [13]:
from soda.scan import Scan

scan = Scan()
scan.disable_telemetry()
scan.set_data_source_name('snapshot')
scan.add_spark_session(spark, 'snapshot')

scan.add_sodacl_yaml_str('''
checks for snapshot:
  - row_count = 13
  - duplicate_count(id, mail) = 0
  - missing_count(id, mail) = 0
  - invalid_count(mail) = 0:
      valid format: email
  - invalid_percent(ssn, birthdate) = 0:
      valid length: 10
discover tables:
  tables:
    - include snapshot
profile columns:
  columns:
    - snapshot.%
''')
# scan.set_verbose() # ?
exit_code = scan.execute()
exit_code                       

root
 |-- count(1): long (nullable = false)
 |-- count(CASE WHEN (id IS NULL) THEN 1 END): long (nullable = false)
 |-- count(CASE WHEN ((NOT (mail IS NULL)) AND (NOT RLIKE(mail, ^[a-zA-ZÀ-ÿĀ-ſƀ-ȳ0-9.\-_%]+@[a-zA-ZÀ-ÿĀ-ſƀ-ȳ0-9.\-_%]+\.[A-Za-z]{2,4}$))) THEN 1 END): long (nullable = false)
 |-- count(1): long (nullable = false)
 |-- count(CASE WHEN ((NOT (ssn IS NULL)) AND (NOT (length(ssn) = 10))) THEN 1 END): long (nullable = false)

+--------+----------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------------------------------------------------------------------+
|count(1)|count(CASE WHEN (id IS NULL) THEN 1 END)|count(CASE WHEN ((NOT (mail IS NULL)) AND (NOT RLIKE(mail, ^[a-zA-ZÀ-ÿĀ-ſƀ-ȳ0-9.\-_%]+@[a-zA-ZÀ-ÿĀ-ſƀ-ȳ0-9.\-_%]+\.[A-Za-z]{2,4}$))) THEN 1 END)|count(1)|count(CASE WHEN ((NOT (ssn IS NULL)) AND (NOT (length(ssn) = 10))) THEN

2

### Store soda scan results to delta table

In [15]:
for ele in scan._checks:
    print(ele.get_cloud_dict())
pd.DataFrame([ele.get_cloud_dict() for ele in scan._checks])

{'identity': '27c242a2', 'name': 'row_count = 13', 'type': 'metricThreshold', 'definition': 'checks for snapshot:\n  row_count = 13', 'location': {'filePath': 'sodacl_string.yml', 'line': 3, 'col': 5}, 'dataSource': 'snapshot', 'table': 'snapshot', 'column': None, 'metrics': ['metric-snapshot-snapshot-row_count'], 'outcome': 'fail', 'diagnostics': {'value': 14, 'fail': {'greaterThan': 13.0, 'lessThan': 13.0}}}
{'identity': '9027c3d2', 'name': 'duplicate_count(id, mail) = 0', 'type': 'metricThreshold', 'definition': 'checks for snapshot:\n  duplicate_count(id, mail) = 0', 'location': {'filePath': 'sodacl_string.yml', 'line': 4, 'col': 5}, 'dataSource': 'snapshot', 'table': 'snapshot', 'column': None, 'metrics': ['metric-snapshot-snapshot-duplicate_count-248c8fa3'], 'outcome': 'pass', 'diagnostics': {'value': 0, 'fail': {'greaterThan': 0.0, 'lessThan': 0.0}}}
{'identity': 'c322a794', 'name': 'missing_count(id, mail) = 0', 'type': 'metricThreshold', 'definition': 'checks for snapshot:\n  

,identity,name,type,definition,location,dataSource,table,column,metrics,outcome,diagnostics
0,27c242a2,row_count = 13,metricThreshold,checks for snapshot:\n row_count = 13,"{'filePath': 'sodacl_string.yml', 'line': 3, '...",snapshot,snapshot,None,[metric-snapshot-snapshot-row_count],fail,"{'value': 14, 'fail': {'greaterThan': 13.0, 'l..."
1,9027c3d2,"duplicate_count(id, mail) = 0",metricThreshold,"checks for snapshot:\n duplicate_count(id, ma...","{'filePath': 'sodacl_string.yml', 'line': 4, '...",snapshot,snapshot,None,[metric-snapshot-snapshot-duplicate_count-248c...,pass,"{'value': 0, 'fail': {'greaterThan': 0.0, 'les..."
2,c322a794,"missing_count(id, mail) = 0",metricThreshold,"checks for snapshot:\n missing_count(id, mail...","{'filePath': 'sodacl_string.yml', 'line': 5, '...",snapshot,snapshot,id,[metric-snapshot-snapshot-id-missing_count-248...,pass,"{'value': 0, 'fail': {'greaterThan': 0.0, 'les..."
3,55ab369a,invalid_count(mail) = 0,metricThreshold,checks for snapshot:\n - invalid_count(mail) ...,"{'filePath': 'sodacl_string.yml', 'line': 6, '...",snapshot,snapshot,mail,[metric-snapshot-snapshot-mail-invalid_count-6...,pass,"{'value': 0, 'fail': {'greaterThan': 0.0, 'les..."
4,2b0efc8f,"invalid_percent(ssn, birthdate) = 0",metricThreshold,"checks for snapshot:\n - invalid_percent(ssn,...","{'filePath': 'sodacl_string.yml', 'line': 8, '...",snapshot,snapshot,ssn,[metric-snapshot-snapshot-ssn-invalid_percent-...,fail,"{'value': 100.0, 'fail': {'greaterThan': 0.0, ..."


In [16]:
for ele in scan._metrics:
    print(ele.get_cloud_dict())
pd.DataFrame([ele.get_cloud_dict() for ele in scan._metrics])

{'identity': 'metric-snapshot-snapshot-duplicate_count-248c8fa3', 'metricName': 'duplicate_count', 'value': 0}
{'identity': 'metric-snapshot-snapshot-ssn-invalid_percent-3a74e0ac', 'metricName': 'invalid_percent', 'value': 100.0}
{'identity': 'metric-snapshot-snapshot-row_count-0496d604', 'metricName': 'row_count', 'value': 14}
{'identity': 'metric-snapshot-snapshot-ssn-invalid_count-3a74e0ac', 'metricName': 'invalid_count', 'value': 14}
{'identity': 'metric-snapshot-snapshot-row_count', 'metricName': 'row_count', 'value': 14}
{'identity': 'metric-snapshot-snapshot-mail-invalid_count-61813b33', 'metricName': 'invalid_count', 'value': 0}
{'identity': 'metric-snapshot-snapshot-id-missing_count-248c8fa3', 'metricName': 'missing_count', 'value': 0}


,identity,metricName,value
0,metric-snapshot-snapshot-duplicate_count-248c8fa3,duplicate_count,0.0
1,metric-snapshot-snapshot-ssn-invalid_percent-3...,invalid_percent,100.0
2,metric-snapshot-snapshot-row_count-0496d604,row_count,14.0
3,metric-snapshot-snapshot-ssn-invalid_count-3a7...,invalid_count,14.0
4,metric-snapshot-snapshot-row_count,row_count,14.0
5,metric-snapshot-snapshot-mail-invalid_count-61...,invalid_count,0.0
6,metric-snapshot-snapshot-id-missing_count-248c...,missing_count,0.0


In [17]:
import pandas as pd
for ele in scan._profile_columns_result_tables:
    print(ele.get_cloud_dict())
display(pd.DataFrame(ele.get_cloud_dict()))
display(pd.DataFrame(scan._profile_columns_result_tables[0].get_cloud_dict()['columnProfiles']))

{'table': 'snapshot', 'dataSource': 'snapshot', 'columnProfiles': [{'columnName': 'id', 'profile': {'mins': [0.0, 1.0, 3.0, 4.0, 6.0], 'maxs': [19.0, 18.0, 17.0, 16.0, 15.0], 'min': 0.0, 'max': 19.0, 'frequent_values': [{'value': '12', 'frequency': 1}, {'value': '1', 'frequency': 1}, {'value': '13', 'frequency': 1}, {'value': '6', 'frequency': 1}, {'value': '16', 'frequency': 1}, {'value': '3', 'frequency': 1}, {'value': '19', 'frequency': 1}, {'value': '15', 'frequency': 1}, {'value': '9', 'frequency': 1}, {'value': '17', 'frequency': 1}], 'avg': 10.5, 'sum': 147.0, 'stddev': 6.583779601881721, 'variance': 43.34615384615384, 'distinct': 14, 'missing_count': 0, 'histogram': {'boundaries': [0.0, 1.46, 2.92, 4.38, 5.85, 7.31, 8.77, 10.23, 11.69, 13.15, 14.62, 16.08, 17.54, 19.0], 'frequencies': [2, 0, 2, 0, 1, 0, 1, 0, 2, 1, 2, 1, 1, 1]}, 'avg_length': None, 'min_length': None, 'max_length': None}}, {'columnName': 'job', 'profile': {'mins': None, 'maxs': None, 'min': None, 'max': None, '

,table,dataSource,columnProfiles
0,snapshot,snapshot,"{'columnName': 'id', 'profile': {'mins': [0.0,..."
1,snapshot,snapshot,"{'columnName': 'job', 'profile': {'mins': None..."
2,snapshot,snapshot,"{'columnName': 'company', 'profile': {'mins': ..."
3,snapshot,snapshot,"{'columnName': 'ssn', 'profile': {'mins': None..."
4,snapshot,snapshot,"{'columnName': 'website', 'profile': {'mins': ..."
5,snapshot,snapshot,"{'columnName': 'username', 'profile': {'mins':..."
6,snapshot,snapshot,"{'columnName': 'name', 'profile': {'mins': Non..."
7,snapshot,snapshot,"{'columnName': 'address', 'profile': {'mins': ..."
8,snapshot,snapshot,"{'columnName': 'mail', 'profile': {'mins': Non..."
9,snapshot,snapshot,"{'columnName': 'birthdate', 'profile': {'mins'..."


,columnName,profile
0,id,"{'mins': [0.0, 1.0, 3.0, 4.0, 6.0], 'maxs': [1..."
1,job,"{'mins': None, 'maxs': None, 'min': None, 'max..."
2,company,"{'mins': None, 'maxs': None, 'min': None, 'max..."
3,ssn,"{'mins': None, 'maxs': None, 'min': None, 'max..."
4,website,"{'mins': None, 'maxs': None, 'min': None, 'max..."
5,username,"{'mins': None, 'maxs': None, 'min': None, 'max..."
6,name,"{'mins': None, 'maxs': None, 'min': None, 'max..."
7,address,"{'mins': None, 'maxs': None, 'min': None, 'max..."
8,mail,"{'mins': None, 'maxs': None, 'min': None, 'max..."
9,birthdate,"{'mins': None, 'maxs': None, 'min': None, 'max..."
